# Task 2: General Hartree-Fock solver

Having set up an initial single-particle basis set $\{\chi_{\alpha}\}_{\alpha = 1}^{l}$ we now set out to find the ground state of the two-body fermion problem.
The full time-independent Hamiltonian used by {cite}`zanghellini_2004` is given by
\begin{align}
    \hat{H}
    = \hat{h} + \hat{u}
    = \sum_{i = 1}^{n}\left(
        -\frac{1}{2}\frac{\text{d}^2}{\text{d}x^2}
        + \frac{1}{2}\omega^2 x^2
    \right)
    + \sum_{i = 1}^{n} \sum_{j = i + 1}^{n} \frac{\alpha}{\sqrt{(x_i - x_j)^2 + a^2}},
\end{align}
where the number of particles $n = 2$, $a = 0.25$, $\alpha = 1$ (not to be confused with the index $\alpha$ in the basis of harmonic oscillator eigenstates), and $\omega = 0.25$.

We start from the time-independent Schrödinger equation
\begin{align}
    \hat{H} | \Psi \rangle = E | \Psi \rangle,
\end{align}
where $| \Psi \rangle$ is a many-body wave function.
We know that in the non-interacting case the exact ground state of the Hamiltonian will be a single Slater determinant (which we denote $| \Phi \rangle$) of the lowest single-particle eigenstates.
This motivates the ansatz of using a single Slater determinant for the full many-body problem
\begin{align}
    | \Psi \rangle \approx | \Phi \rangle.
\end{align}
Now, the goal of the Hartree-Fock method is to find the single Slater determinant that minimizes the energy, i.e., gets closest to the "true" many-body ground state energy $E_{gs}$ of the problem.
This is formulated via the variational principle
\begin{align}
    E_{gs} \leq E[\Psi] = \langle \Psi | \hat{H} | \Psi \rangle,
\end{align}
for any normalized wave function $| \Psi \rangle$.
There is also an extra requirement on the Hartree-Fock wave function, and that is that the Hartree-Fock orbitals (which we denote $\{\phi_i\}_{i = 1}^{k}$) should be orthonormal, i.e., $\langle \phi_i | \phi_j \rangle = \delta_{ij}$.
To include this requirement in the variational principle we use Lagrange's method of undetermined multipliers

$$
    L[\Psi, \boldsymbol{\lambda}]
    = E[\Psi]
    - \lambda_{ji} \left(
        \langle \phi_i | \phi_j \rangle
        - \delta_{ij}
    \right),
$$ (eq:hf-lagrangian)
where $\boldsymbol{\lambda}$ are the Lagrange multipliers.
To avoid confusion with any other Slater determinants we will introduce the notation $| \text{HF} \rangle = |\phi_1\dots\phi_n\rangle$ for the optimized Hartree-Fock Slater determinant.

```{note}
Add derivation of the (canonical) GHF-equations.
```

Minimization of the Lagrangian in {eq}`eq:hf-lagrangian` yields the Hartree-Fock eigenvalue equation

$$
    \hat{f} | \phi_p \rangle = \varepsilon_p | \phi_p \rangle,
$$ (eq:canonical-hf-equation)
where $\hat{f}$ is the single-particle Fock-operator.
Note that this equation is not limited to the occupied states (denoted $\{ \phi_i \}_{i = 1}^n$) in $|\text{HF}\rangle$, but to all states (denoted $\{ \phi_p \}_{p = 1}^k$).
The matrix elements of the single-particle Fock operator are given by
\begin{align}
    \langle \psi | \hat{f} | \xi \rangle
    = \langle \psi | \hat{h} | \xi \rangle
    + \sum_{i = 1}^{n} \langle \psi \phi_i | \hat{u} | \xi \phi_i \rangle
    - \sum_{i = 1}^{n} \langle \psi \phi_i | \hat{u} | \phi_i \xi \rangle,
\end{align}
where $|\psi\rangle$ and $|\xi \rangle$ are two arbitrary single-particle states.

## Types of orbital systems

In quantum chemistry it is common to call the initial single-particle functions for _atomic orbitals_ (AO).
We will also adhere to this convention, thus making the basis of harmonic oscillator single-particle functions the atomic orbitals of our system.
Now, as the harmonic oscillator Hamiltonian is independent of spin we have restricted our view to the spatial part of the atomic orbitals when using quantum-systems to construct our basis elements.
However, when moving to the full fermionic many-body picture (two = many) we have to include spin to get the full picture.
We often talk about three kinds of orbitals depending on the restrictions put on spin.

1. The general spin-orbital is on the form
    \begin{align}
        \phi(x, m_s)
        &= \varphi_1(x) \alpha(m_s) + \varphi_2(x) \beta(m_s),
    \end{align}
    where $\varphi_i(x)$ are the spatial oribtals, and $\sigma \in \{\alpha, \beta\}$ are spin basis functions with $m_s \in \{\uparrow, \downarrow\}$ the spin quantum number.
    Typical choices for the spin basis functions are
    \begin{gather}
        \alpha(\uparrow) = 1, \quad \beta(\uparrow) = 0, \\
        \alpha(\downarrow) = 0, \quad \beta(\downarrow) = 1.
    \end{gather}
2. The unrestricted spin-orbital is on the form
    \begin{align}
        \phi(x, m_s)
        = \varphi_{\sigma}(x) \sigma(m_s),
    \end{align}
    where an unrestricted spin-orbital has a definite spin-direction (either $\alpha$ or $\beta$), but allows for the spatial part to be different between spin-directions.
3. The restricted spin-orbital is on the form
    \begin{align}
        \phi(x, m_s)
        = \varphi(x) \sigma(m_s),
    \end{align}
    where the spin-orbitals have a definite spin-direction with the spatial part equal between the two spin-directions.

We will in this project create a general Hartree-Fock (GHF) solver which implies that we use general spin-orbitals.
This will turn out to be easiest mathematically (when deriving the equations for GHF) and the most general formulation.
However, the restricted and unrestricted formulations are more numerically efficient.

## Adding spin to the harmonic oscillator basis

From [task-1](task_1_mel) we have constructed a spatial basis of harmonic oscillator eigenstates $\{\chi_{\alpha}\}_{\alpha = 1}^{l}$, with spin-independent matrix elements $h_{\alpha \beta}$, $u^{\alpha \beta}_{\gamma \delta}$, etc.
We now wish to include spin in the harmonic oscillator basis functions.
The way we do this is by doubling each spatial basis function (and hence each axis in the matrix elements) essentially creating duplicates.

We will denote this basis of general harmonic oscillator spin-orbitals by $\{\psi_{\mu}\}_{\mu = 1}^{2l}$.
The question is then how we can go from $h_{\alpha \beta} \to h_{\mu \nu}$ and similarly for the other matrix elements.
We let the general harmonic oscillator spin-orbitals be given by
\begin{align}
    |\psi_{\mu}\rangle
    &= |\chi_{\alpha}\rangle \otimes |\sigma\rangle,
\end{align}
where $\alpha = \mu // 2$ (integer division), and $\sigma = \alpha$ if $\mu$ is even and $\sigma = \beta$ if $\mu$ is odd.
The one-body Hamiltonian matrix elements are thus given by
\begin{align}
    h_{\mu \nu}
    &= \langle \psi_{\mu} | \hat{h} \otimes \hat{I} | \psi_{\nu} \rangle
    = \left(\langle \chi_{\alpha} | \otimes \langle \sigma | \right)
    \hat{h} \otimes \hat{I} \left( | \chi_{\beta} \rangle \otimes | \tau \rangle \right)
    \\
    &= \langle \chi_{\alpha} | \hat{h} | \chi_{\beta} \rangle
    \langle \sigma | \tau \rangle
    = h_{\alpha \beta} \delta_{\sigma \tau},
\end{align}
where $\hat{I}$ is the spin-basis identity operator, and we've chosen an orthonormal basis of spin states.
The full general spin-orbital matrix $\mathbf{h}_{GSO}$ can then be constructed from the (spatial) atomic orbital matrix $\mathbf{h}_{SpaS}$ and $\mathbf{I}_{2\times 2}$ the $2 \times 2$ identity matrix by
\begin{align}
    \mathbf{h}_{GSO} = \mathbf{h}_{SpaS} \otimes \mathbf{I}_{2 \times 2},
\end{align}
where $\otimes$ is the Kronecker product.
This ensures that all even (odd) indices correspond to $\alpha$ ($\beta$) spin-states.
From [quantum-systems](https://github.com/Schoyen/quantum-systems) this is done automatically when passing in a `BasisSet` to the class `GeneralOrbitalSystem`.
Below we demonstrate how this is done for harmonic oscillator basis set.

In [1]:
%reload_ext nb_black

<IPython.core.display.Javascript object>

In [14]:
import numpy as np
import matplotlib.pyplot as plt

from quantum_systems import ODQD, GeneralOrbitalSystem

<IPython.core.display.Javascript object>

In [15]:
n = 2  # Number of particles
l = 10  # Number of eigenstates
grid_length = 10  # The width of the one-dimensional grid
num_grid_points = 2001  # The number of discretized points on the grid.
# More points give better results for the single-particle basis at the cost of slower setup.

alpha = 1  # The strength of the Coulomb interaction
a = 0.25  # The shielding parameter in the Coulomb interaction potential

omega = 0.25  # The frequency of the harmonic oscillator trap

<IPython.core.display.Javascript object>

In [16]:
potential = ODQD.HOPotential(omega=omega)  # Harmonic oscillator potential

<IPython.core.display.Javascript object>

In [31]:
odqd = ODQD(
    l=l,
    grid_length=grid_length,
    num_grid_points=num_grid_points,
    alpha=alpha,
    a=a,
    potential=potential,
)

print(f"Basis l: {odqd.l}")
print(f"Basis grid: {odqd.grid.shape}")
print(f"Basis h.shape: {odqd.h.shape}")
print(f"Basis u.shape: {odqd.u.shape}")
print(f"Basis x.shape: {odqd.position.shape}")
print(f"Basis spf.shape: {odqd.spf.shape}")

Basis l: 10
Basis grid: (2001,)
Basis h.shape: (10, 10)
Basis u.shape: (10, 10, 10, 10)
Basis x.shape: (1, 10, 10)
Basis spf.shape: (10, 2001)


<IPython.core.display.Javascript object>

In [32]:
# This adds spin to all matrix elements
# Note that gos.l = 2 * l
# Note also that odqd itself is transformed into a general orbital system.
# If you want to keep the pure spatial basis you can pass in a copy, viz.
# gos = GeneralOrbitalSystem(n, gos.copy_basis())
gos = GeneralOrbitalSystem(n, odqd)

print(f"GOS l: {gos.l}")
print(f"GOS grid: {gos._basis_set.grid.shape}")
print(f"GOS h.shape: {gos.h.shape}")
print(f"GOS u.shape: {gos.u.shape}")
print(f"GOS x.shape: {gos.position.shape}")
print(f"GOS spf.shape: {gos.spf.shape}")

GOS l: 20
GOS grid: (2001,)
GOS h.shape: (20, 20)
GOS u.shape: (20, 20, 20, 20)
GOS x.shape: (1, 20, 20)
GOS spf.shape: (20, 2001)


<IPython.core.display.Javascript object>

Looking at the output above we see that the number of basis elements $l$ have been doubled.
However, the grid and the number of dimensions (the first axis in `gos.position`) are unchanged.
It is worth pointing out that `gos.spf[::2]` (all even indices) correspond to the spatial part of the $\alpha$ component of a general spin-orbital and `gos.spf[1::2]` (all odd indices) the $\beta$ component.

## The Roothan-Hall equations

The way we solve the Hartree-Fock equation in {eq}`eq:canonical-hf-equation` is by expanding the Hartree-Fock orbitals (also called molecular orbitals (MO)) $\{ \phi_p \}_{p = 1}^k$ in an atomic orbital basis $\{ \psi_{\mu} \}_{\mu = 1}^{l}$ (which we here take to be the harmonic oscillator basis states with spin-doubling), and then projecting the equations onto the known basis set.
We express a basis transformation from the AO basis to the MO basis by

$$
    |\phi_p\rangle = \sum_{\mu = 1}^{l} C_{\mu p} | \psi_{\mu} \rangle,
$$ (eq:basis-transformation)
where the coefficients $C_{\mu p}$ now become our primary unknowns in the Hartree-Fock method.
Projecting the AO basis onto equation {eq}`eq:canonical-hf-equation` we have
\begin{gather}
    \langle \psi_{\mu} | \hat{f} | \phi_p \rangle
    = \varepsilon_{p} \langle \psi_{\mu} | \phi_p \rangle
    \implies
    \sum_{\nu = 1}^{l} f_{\mu \nu} C_{\nu p}
    = \varepsilon_{p} \sum_{\nu = 1}^{l} \delta_{\mu \nu} C_{\nu p}
    =  C_{\mu p} \varepsilon_{p}
    \\
    \implies
    \mathbf{F} \mathbf{C}
    = \mathbf{C} \boldsymbol{\varepsilon},
\end{gather}
where $[\mathbf{F}]_{\mu \nu} = f_{\mu \nu} = \langle \psi_{\mu} | \hat{f} | \psi_{\nu} \rangle$ are the Fock matrix elements in the AO basis, $[\mathbf{C}]_{\mu p} = C_{\mu p}$ the basis transformation coefficient matrix, and $\boldsymbol{\varepsilon} = [\varepsilon_1, \dots, \varepsilon_k]^T$ the vector of Hartree-Fock single-particle eigenenergy (not to be confused with the full many-body energy).
The last eigenvalue equation is known as the Roothan-Hall equations.
Here both $\mathbf{C}$ and $\boldsymbol{\varepsilon}$ are the primary unknowns.